In [68]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from statsmodels.tsa.stattools import coint
from itertools import combinations

# Atsisiunciame duomenis ( pvz . NVDA ir BTC )
tickers = ["BNB-USD", "DOGE-USD", 'KC=F', 'SB=F']
df = yf.download(tickers, start="2018-01-01")
df = df.ffill().bfill() # Uzpildome praleistas reiksmes, kadangi dalis musu portfelio yra derivatyvai
returns = df["Close"].pct_change().dropna()


[*********************100%***********************]  4 of 4 completed


In [48]:
print(returns)
print()
print(returns.mean())

Ticker       BNB-USD  DOGE-USD      KC=F      SB=F
Date                                              
2018-01-02  0.050289  0.026490  0.000000  0.000000
2018-01-03  0.078992  0.019136 -0.012289 -0.001305
2018-01-04 -0.033756  0.034764  0.007387 -0.003919
2018-01-05  0.618973  0.261613 -0.008491 -0.011148
2018-01-06  0.525742  0.221583  0.000000  0.000000
...              ...       ...       ...       ...
2026-02-19  0.004361 -0.002264 -0.000522 -0.007057
2026-02-20  0.030476  0.018649  0.003132  0.016347
2026-02-21  0.000375 -0.016430  0.000000  0.000000
2026-02-22 -0.018360 -0.030717  0.000000  0.000000
2026-02-23 -0.036944 -0.027181 -0.034859 -0.020280

[2975 rows x 4 columns]

Ticker
BNB-USD     0.002585
DOGE-USD    0.003361
KC=F        0.000423
SB=F        0.000073
dtype: float64


In [58]:
import plotly.io as pio
pio.renderers.default = "notebook" 

In [72]:
coint( df['Close']['KC=F'], df['Close']['SB=F'] )

(np.float64(-1.2451056614392801),
 np.float64(0.8456907049925528),
 array([-3.9001251 , -3.33818459, -3.04587592]))

In [79]:
risk_free_rate = 0.03  # Tarkime , JAV obligaciju palukanos 3%


def analyze_asset(df, weights: list[float]):
    """
    Kainos grafiką su indikatoriumi. - Candlestick
    95% VaR reikšmę.
    Expected Shortfall
    Sharpe koeficientą.
    """

    assert np.isclose( sum(weights), 1.0), "Svoriai turi susisumuoti i 1"
    assert len(weights) == df['Close'].shape[1],\
        f"Svoriai turi atitikti instrumentus, {len(weights)}, {df.shape[1]}"

    print("Viso portfelio analizė")
    print("-"*20)

    # Apskaiciuoti bendras portfelio vertes
    returns = df['Close']\
        .apply( lambda x: np.dot(x, weights), axis=1)\
        .pct_change()\
        .dropna()

    returns_individual = df['Close'].pct_change().dropna()

    print( np.corrcoef( returns_individual.T ) )

    print( "Vidutinė grąža:", returns.mean() )

    annual_return = returns.mean() * 365
    annual_std = returns.std() * np.sqrt(365)

    sharpe_ratio = (annual_return - risk_free_rate) / annual_std
    print(f"Šarpo rodiklis: { sharpe_ratio :.2f} ")

    # Apskaiciuoti VaR atskiriems 
    var = np.percentile ( returns , 5)
    print( f'Istorinis VaR 95 %: {var:.2f}' )

    # Apskaiciuoti ES
    es = returns [ returns <= var ].mean()
    print( f'Expected Shortfall ( ES ) 95 %: {es:.2f}' )

    cointegrated_pairs = []

    # Iterate over all unique pairs of columns
    close_prices = df['Close']
    for col1, col2 in combinations(close_prices.columns, 2):
        score, p_value, _ = coint(close_prices[col1], close_prices[col2])
        if p_value < 0.05:
            cointegrated_pairs.append((col1, col2))
            print(f"Cointegrated pair: {col1} & {col2} (p-value={p_value:.4f})")
    
    if not cointegrated_pairs:
        print("No cointegrated pairs found.")


    # Analizuoti kiekviena atskirai
    for ticker in tickers:
        print('\n', ticker, 'analizė')
        print( '-' * 20 )

        df_plot = df

        if isinstance(df_plot.columns, pd.MultiIndex):
            df_plot = df_plot.xs(ticker, axis=1, level=1)

        fig = go.Figure(
            data=[
                go.Candlestick(
                    x=df_plot.index,
                    open=df_plot["Open"],
                    high=df_plot["High"],
                    low=df_plot["Low"],
                    close=df_plot["Close"],
                    name="Market Data",
                )
            ]
        )
        fig.update_layout(
            title=f"{ticker} interaktyvus grafikas",
            yaxis_title="Kaina",
            xaxis_title="Data",
        )
        fig.show()

        returns = df["Close"][ticker].pct_change().dropna()
        print( "Vidutinė grąža:", returns.mean())

        if "-USD" in ticker: # Jeigu kriptovaliuta, tuomet prekyba vyksta visus metus
            annual_return = returns.mean() * 365
            annual_std = returns.std() * np.sqrt(365)
        else:
            annual_return = returns.mean() * 252
            annual_std = returns.std() * np.sqrt(252)

        sharpe_ratio = (annual_return - risk_free_rate) / annual_std
        print(f"Šarpo rodiklis: { sharpe_ratio :.2f} ")

        # Apskaiciuoti VaR atskiriems 
        var = np.percentile ( returns , 5)
        print( f'Istorinis VaR 95 %: {var:.2f}' )

        # Apskaiciuoti ES
        es = returns [ returns <= var ].mean()
        print( f'Expected Shortfall ( ES ) 95 %: {es:.2f}' )


analyze_asset(df, [0.5, 0.1, 0.2, 0.2])


Viso portfelio analizė
--------------------
[[1.         0.29029463 0.03264952 0.07594329]
 [0.29029463 1.         0.0097543  0.03048585]
 [0.03264952 0.0097543  1.         0.1672791 ]
 [0.07594329 0.03048585 0.1672791  1.        ]]
Vidutinė grąža: 0.0012294416321508126
Šarpo rodiklis: 0.73 
Istorinis VaR 95 %: -0.04
Expected Shortfall ( ES ) 95 %: -0.06
Cointegrated pair: BNB-USD & KC=F (p-value=0.0040)
Cointegrated pair: DOGE-USD & KC=F (p-value=0.0036)
Cointegrated pair: DOGE-USD & SB=F (p-value=0.0355)

 BNB-USD analizė
--------------------


Vidutinė grąža: 0.002584752096086762
Šarpo rodiklis: 0.98 
Istorinis VaR 95 %: -0.06
Expected Shortfall ( ES ) 95 %: -0.10

 DOGE-USD analizė
--------------------


Vidutinė grąža: 0.003360823349725169
Šarpo rodiklis: 0.69 
Istorinis VaR 95 %: -0.08
Expected Shortfall ( ES ) 95 %: -0.12

 KC=F analizė
--------------------


Vidutinė grąža: 0.00042283745662907826
Šarpo rodiklis: 0.26 
Istorinis VaR 95 %: -0.03
Expected Shortfall ( ES ) 95 %: -0.04

 SB=F analizė
--------------------


Vidutinė grąža: 7.272155327976163e-05
Šarpo rodiklis: -0.05 
Istorinis VaR 95 %: -0.02
Expected Shortfall ( ES ) 95 %: -0.03
